<a href="https://colab.research.google.com/github/stazam/M7DataSp---Advanced-data-science-techniques/blob/main/HocekyBetting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import files
uploaded = files.upload()

Saving hockey_datas.csv to hockey_datas.csv


In [17]:
df = pd.read_csv("hockey_datas.csv")

In [4]:
df.shape

(1396, 23)

In [5]:
df.head()

,Unnamed: 0,id,game_number,date,home,away,place,viewers,win_periods_home,draw_periods,win_periods_away,score_home,score_away,shots_home,shots_away,saves_home,saves_away,powerplay_home,powerplay_away,weakening_home,weakening_away,faceoff_home,faceoff_away
0,0,83430,1,11/09/2015,"HK Poprad, s.r.o.",HKM a.s. Zvolen,ZŠ Poprad,1049,2,0,1,3,2,0.1000,0.0769,0.9231,0.9000,0.3333,0.0000,1.0000,0.6667,0.0,0.0
1,1,83431,2,11/09/2015,"HK DUKLA Trenčín, a.s.",MHC MOUNTFIELD Martin,"ZŠ Pavla Demitru, Trenčín",1630,2,1,0,3,1,0.0732,0.0294,0.9706,0.9268,0.5000,0.0000,1.0000,0.5000,0.0,0.0
2,2,83432,3,11/09/2015,"HK 36 Skalica, s.r.o.","ŠHK 37 Piešťany, s.r.o.",HANT arena Skalica,1086,0,2,1,3,5,0.0938,0.1190,0.8810,0.9063,0.0000,0.3333,0.6667,1.0000,0.0,0.0
3,3,83433,4,11/09/2015,HC Košice s.r.o.,MsHK DOXXbet Žilina,STEEL Arena Košice,2548,2,1,1,5,4,0.1250,0.1667,0.8333,0.8750,0.2000,0.7500,0.2500,0.8000,0.0,0.0
4,4,83434,5,10/09/2015,HC ‘05 iClinic BANSKÁ BYSTRICA,"HK Nitra, a.s.",ZŠ Banská Bystrica,1896,2,0,1,4,1,0.1081,0.0400,0.9600,0.8919,0.0000,0.0000,1.0000,1.0000,0.0,0.0


1. pridat stplpec result - HOTOVO
2. zmeit stlpec date na datetime premennu
3. upravit vsetky hodnoty v stlpcoch
4. pridat sltpce s  premennymi ktore mal ten typek
6. pozot na chybajuce pozorovania pri faceoff, places a skontorlovat aj dalsie a upravit a nahradit medianom alebo takym niecim
7. zistit unikatne hodnoty a podla toho to upravit

conceded = pocet inkasovanych golov

H2H v specifickej sezone a sucasne aj za celkove predchadzajuce obdobie

vitazstva celkovo za vsetky sezony, vitazstva celkovo za danu sezonu proti danemu superovi

In [18]:
def name_parser(x):

  x = x.lower()
  clubs = ['Poprad', 'Trenčín', 'Skalica', 'Košice', 'BANSKÁ BYSTRICA', 'Zvolen', 'Piešťany', 'Martin', 'Nitra', 'Žilina', 
           'Liptovský Mikuláš', 'Nové Zámky', 'Detva', 'Újbuda', 'Miskolc', 'Michalovce', 'Bratislava']

  clubs = [x.lower() for x in clubs]
  for club in clubs:
    if club in x:
      return club
  
  return 

df['home'] = df['home'].apply(name_parser)
df['away'] = df['away'].apply(name_parser)

In [19]:
#mozno by som este mohol pridat to, ze ked nie je domaci tym na domacom lade, tak ako sa to premietne do vysledku -- ale to sa stava v minime pripadov
import datetime

df['date'] = pd.to_datetime(df['date'])


In [21]:
df.loc[df['score_home'] > df['score_away'], 'result'] = 'home'
df.loc[df['score_home'] < df['score_away'], 'result'] = 'away'
df.loc[df['win_periods_home'] + df['draw_periods'] + df['win_periods_away'] >= 4, 'result'] = 'draw'

In [22]:
def season(x):

  if x >= 83430 and x <= 83699:
    return "2015/2016"
  elif x >= 87717 and x <= 87986:
    return "2016/2017"
  elif x >= 96815 and x <= 97084:
    return "2017/2018"
  elif x >= 101458 and x <= 101787:
    return "2018/2019"
  else:
    return "2019/2020"

df['season'] = df['id'].apply(season)

In [23]:
df

,Unnamed: 0,id,game_number,date,home,away,place,viewers,win_periods_home,draw_periods,win_periods_away,score_home,score_away,shots_home,shots_away,saves_home,saves_away,powerplay_home,powerplay_away,weakening_home,weakening_away,faceoff_home,faceoff_away,result,season
0,0,83430,1,2015-11-09,poprad,zvolen,ZŠ Poprad,1049,2,0,1,3,2,0.1000,0.0769,0.9231,0.9000,0.3333,0.0000,1.0000,0.6667,0.0000,0.0000,home,2015/2016
1,1,83431,2,2015-11-09,trenčín,martin,"ZŠ Pavla Demitru, Trenčín",1630,2,1,0,3,1,0.0732,0.0294,0.9706,0.9268,0.5000,0.0000,1.0000,0.5000,0.0000,0.0000,home,2015/2016
2,2,83432,3,2015-11-09,skalica,piešťany,HANT arena Skalica,1086,0,2,1,3,5,0.0938,0.1190,0.8810,0.9063,0.0000,0.3333,0.6667,1.0000,0.0000,0.0000,away,2015/2016
3,3,83433,4,2015-11-09,košice,žilina,STEEL Arena Košice,2548,2,1,1,5,4,0.1250,0.1667,0.8333,0.8750,0.2000,0.7500,0.2500,0.8000,0.0000,0.0000,draw,2015/2016
4,4,83434,5,2015-10-09,banská bystrica,nitra,ZŠ Banská Bystrica,1896,2,0,1,4,1,0.1081,0.0400,0.9600,0.8919,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,home,2015/2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1391,1391,110103,358,2020-02-21,miskolc,újbuda,"Ice rink Miskolc, HUN",1385,1,0,2,1,3,0.0222,0.1000,0.9259,0.9778,0.0000,0.2500,0.7500,1.0000,0.5574,0.4426,away,2019/2020
1392,1392,110105,360,2020-02-21,košice,nové zámky,STEEL Arena Košice,7067,2,1,0,7,2,0.1892,0.0952,0.9048,0.8108,0.1429,0.2500,0.7500,0.8571,0.5536,0.4464,home,2019/2020
1393,1393,110106,361,2020-02-21,trenčín,poprad,"ZŠ Pavla Demitru, Trenčín",3120,1,0,3,3,4,0.0000,0.1212,0.8788,0.9143,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,draw,2019/2020
1394,1394,110107,362,2020-02-21,bratislava,liptovský mikuláš,Arena Ondreja Nepelu,4697,3,0,0,11,0,0.2500,0.0000,1.0000,0.7500,0.5000,0.0000,1.0000,0.5000,0.5893,0.4107,home,2019/2020


In [24]:
df_home = df.copy()  
df_away = df.copy()

In [25]:
df_home.columns = df_home.columns.str.replace("home", "team")
df_home.columns = df_home.columns.str.replace("away", "opponent")
df_home = df_home.replace('home', 'win')
df_home = df_home.replace('away', 'lost')
df_home['h_a'] = 'home'

In [26]:
df_home

,Unnamed: 0,id,game_number,date,team,opponent,place,viewers,win_periods_team,draw_periods,win_periods_opponent,score_team,score_opponent,shots_team,shots_opponent,saves_team,saves_opponent,powerplay_team,powerplay_opponent,weakening_team,weakening_opponent,faceoff_team,faceoff_opponent,result,season,h_a
0,0,83430,1,2015-11-09,poprad,zvolen,ZŠ Poprad,1049,2,0,1,3,2,0.1000,0.0769,0.9231,0.9000,0.3333,0.0000,1.0000,0.6667,0.0000,0.0000,win,2015/2016,home
1,1,83431,2,2015-11-09,trenčín,martin,"ZŠ Pavla Demitru, Trenčín",1630,2,1,0,3,1,0.0732,0.0294,0.9706,0.9268,0.5000,0.0000,1.0000,0.5000,0.0000,0.0000,win,2015/2016,home
2,2,83432,3,2015-11-09,skalica,piešťany,HANT arena Skalica,1086,0,2,1,3,5,0.0938,0.1190,0.8810,0.9063,0.0000,0.3333,0.6667,1.0000,0.0000,0.0000,lost,2015/2016,home
3,3,83433,4,2015-11-09,košice,žilina,STEEL Arena Košice,2548,2,1,1,5,4,0.1250,0.1667,0.8333,0.8750,0.2000,0.7500,0.2500,0.8000,0.0000,0.0000,draw,2015/2016,home
4,4,83434,5,2015-10-09,banská bystrica,nitra,ZŠ Banská Bystrica,1896,2,0,1,4,1,0.1081,0.0400,0.9600,0.8919,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,win,2015/2016,home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1391,1391,110103,358,2020-02-21,miskolc,újbuda,"Ice rink Miskolc, HUN",1385,1,0,2,1,3,0.0222,0.1000,0.9259,0.9778,0.0000,0.2500,0.7500,1.0000,0.5574,0.4426,lost,2019/2020,home
1392,1392,110105,360,2020-02-21,košice,nové zámky,STEEL Arena Košice,7067,2,1,0,7,2,0.1892,0.0952,0.9048,0.8108,0.1429,0.2500,0.7500,0.8571,0.5536,0.4464,win,2019/2020,home
1393,1393,110106,361,2020-02-21,trenčín,poprad,"ZŠ Pavla Demitru, Trenčín",3120,1,0,3,3,4,0.0000,0.1212,0.8788,0.9143,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,draw,2019/2020,home
1394,1394,110107,362,2020-02-21,bratislava,liptovský mikuláš,Arena Ondreja Nepelu,4697,3,0,0,11,0,0.2500,0.0000,1.0000,0.7500,0.5000,0.0000,1.0000,0.5000,0.5893,0.4107,win,2019/2020,home


In [27]:
df_away.columns = df_away.columns.str.replace("away", "team")
df_away.columns = df_away.columns.str.replace("home", "opponent")
df_away = df_away.replace('away', 'win')
df_away = df_away.replace('home', 'lost')
df_away['h_a'] = 'away'

In [30]:
df_home.dtypes

Unnamed: 0                       int64
id                               int64
game_number                      int64
date                    datetime64[ns]
team                            object
opponent                        object
place                           object
viewers                          int64
win_periods_team                 int64
draw_periods                     int64
win_periods_opponent             int64
score_team                       int64
score_opponent                   int64
shots_team                     float64
shots_opponent                 float64
saves_team                     float64
saves_opponent                 float64
powerplay_team                 float64
powerplay_opponent             float64
weakening_team                 float64
weakening_opponent             float64
faceoff_team                   float64
faceoff_opponent               float64
result                          object
season                          object
h_a                      

In [31]:
df_teams = pd.concat([df_home, df_away]).reset_index(drop=True)

In [33]:
 df_teams = df_teams.sort_values(['team', 'date']).reset_index(drop=True)

In [35]:
df_teams['win'] = 0
df_teams.loc[df_teams['result'] == 'win', 'win'] = 1
df_teams['draw'] = 0
df_teams.loc[df_teams['result'] == 'draw', 'draw'] = 1
df_teams['lost'] = 0
df_teams.loc[df_teams['result'] == 'lost', 'lost'] = 1   

In [37]:
df_teams['win_R5'] = df_teams.groupby(['team', 'season'])['win'].rolling(5, min_periods=1).mean().values
df_teams['win_R5'] =df_teams.groupby(['team', 'season'])['win_R5'].shift(fill_value=0)

In [40]:
df_teams


,Unnamed: 0,id,game_number,date,team,opponent,place,viewers,win_periods_team,draw_periods,win_periods_opponent,score_team,score_opponent,shots_team,shots_opponent,saves_team,saves_opponent,powerplay_team,powerplay_opponent,weakening_team,weakening_opponent,faceoff_team,faceoff_opponent,result,season,h_a,win,draw,lost,win_R5
0,84,83515,86,2015-01-11,banská bystrica,zvolen,ZŠ Zvolen,3420,0,4,0,4,3,0.1081,0.1034,0.8966,0.8919,0.3333,1.5000,-0.5000,0.6667,0.0000,0.0000,draw,2015/2016,away,0,1,0,0.0
1,39,83469,40,2015-02-10,banská bystrica,žilina,ZŠ Banská Bystrica,1532,0,2,1,0,1,0.0000,0.0000,0.9524,1.0000,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,lost,2015/2016,home,0,0,1,0.0
2,40,83470,41,2015-04-10,banská bystrica,zvolen,ZŠ Banská Bystrica,2816,1,1,2,3,4,0.0750,0.1250,0.8750,0.9250,0.2000,0.0000,1.0000,0.8000,0.0000,0.0000,draw,2015/2016,home,0,1,0,0.0
3,129,83560,131,2015-04-12,banská bystrica,zvolen,ZŠ Banská Bystrica,2841,0,1,2,2,5,0.0714,0.1220,0.8780,0.9286,0.4000,0.1429,0.8571,0.6000,0.0000,0.0000,lost,2015/2016,home,0,0,1,0.0
4,138,83569,140,2015-06-12,banská bystrica,nitra,Nitra Aréna,3355,1,0,2,3,5,0.1000,0.1563,0.8438,0.9000,0.2000,0.2000,0.8000,0.8000,0.0000,0.0000,lost,2015/2016,away,0,0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2787,991,101693,236,2019-03-02,žilina,újbuda,"Ice rink MAC Budapest, HUN",450,1,0,2,2,5,0.0645,0.1389,0.8611,0.9286,0.0000,0.1111,0.8889,1.0000,0.4655,0.5345,lost,2018/2019,away,0,0,1,0.4
2788,980,101682,225,2019-04-01,žilina,banská bystrica,ZŠ Banská Bystrica,2719,0,2,1,3,5,0.1875,0.1613,0.8571,0.8750,0.2500,0.2000,0.8000,0.7500,0.3774,0.6226,lost,2018/2019,away,0,0,1,0.2
2789,922,101624,167,2019-06-01,žilina,trenčín,"ZŠ Pavla Demitru, Trenčín",2069,0,2,1,4,5,0.2000,0.1316,0.8684,0.8000,0.0000,0.0000,1.0000,1.0000,0.1667,0.8333,lost,2018/2019,away,0,0,1,0.2
2790,1046,101748,291,2019-08-01,žilina,banská bystrica,ZŠ Žilina,367,2,0,2,4,5,0.0976,0.0962,0.9038,0.9024,1.0000,1.0000,0.0000,0.0000,0.5263,0.4737,draw,2018/2019,home,0,1,0,0.0


In [13]:
df.loc[ (df['home'] == 'poprad') & (df['away'] == 'zvolen'),:]

,Unnamed: 0,id,game_number,date,home,away,place,viewers,win_periods_home,draw_periods,win_periods_away,score_home,score_away,shots_home,shots_away,saves_home,saves_away,powerplay_home,powerplay_away,weakening_home,weakening_away,faceoff_home,faceoff_away,result,season
0,0,83430,1,2015-11-09,poprad,zvolen,ZŠ Poprad,1049,2,0,1,3,2,0.1000,0.0769,0.9231,0.9000,0.3333,0.0000,1.0000,0.6667,0.0000,0.0000,home,2015/2016
89,89,83520,91,2015-10-11,poprad,zvolen,ZŠ Poprad,1100,1,0,2,3,4,0.1200,0.1333,0.8667,0.8800,0.4000,0.2000,0.8000,0.6000,0.0000,0.0000,away,2015/2016
179,179,83610,181,2016-08-01,poprad,zvolen,ZŠ Poprad,1420,2,0,2,3,4,0.0750,0.1290,0.8710,0.9250,0.2000,0.4000,0.6000,0.8000,0.0000,0.0000,draw,2015/2016
292,292,87749,33,2016-09-25,poprad,zvolen,ZŠ Poprad,1003,1,0,2,3,4,0.0769,0.1429,0.8571,0.9231,0.2500,0.1667,0.8333,0.7500,0.0000,0.0000,away,2016/2017
343,343,87839,123,2016-02-12,poprad,zvolen,ZŠ Poprad,1043,2,1,0,6,1,0.1579,0.0303,0.9697,0.8421,0.2500,0.1667,0.8333,0.7500,0.0000,0.0000,home,2016/2017
431,431,87929,213,2017-03-02,poprad,zvolen,ZŠ Poprad,1062,0,2,1,5,6,0.1471,0.1765,0.8235,0.8529,0.3333,0.2500,0.7500,0.6667,0.0000,0.0000,away,2016/2017
555,555,96883,69,2017-10-22,poprad,zvolen,ZŠ Poprad,1193,0,1,2,1,4,0.0000,0.0000,0.9143,0.9722,0.0000,0.0000,1.0000,1.0000,0.5070,0.4930,away,2017/2018
645,645,96973,159,2017-12-30,poprad,zvolen,ZŠ Poprad,2539,1,1,2,2,3,0.0606,0.0789,0.9412,0.9375,0.0000,0.0000,1.0000,1.0000,0.6000,0.4000,draw,2017/2018
735,735,97063,249,2018-02-25,poprad,zvolen,ZŠ Poprad,1006,1,0,2,3,5,0.0857,0.1852,0.8519,0.9143,0.1667,0.3333,0.6667,0.8333,0.5217,0.4783,away,2017/2018
788,788,101490,33,2018-09-28,poprad,zvolen,ZŠ Poprad,1589,1,2,1,3,2,0.0938,0.0526,0.9512,0.9063,0.0000,0.2000,0.8000,1.0000,0.5469,0.4531,draw,2018/2019


In [ ]:
df['home'] == 'HK Poprad, s.r.o.' and df['away'] == 'HKM a.s. Zvolen'

ValueError: ignored

0

In [ ]:
l = [[1, 2, 3,4], [1, None, 4,5], [2, 1, 3,5], [1, 2, 2,5], [1, 2, 2,5], [1, 2, 2,5]]
df = pd.DataFrame(l, columns=["a", "b", "c","d"])
df

,a,b,c,d
0,1,2.0,3,4
1,1,NaN,4,5
2,2,1.0,3,5
3,1,2.0,2,5
4,1,2.0,2,5
5,1,2.0,2,5


In [ ]:
df.groupby(['a']).shift(fill_value=0)


,a,b,c,d
0,0,0.0,0,0
1,1,2.0,3,4
2,0,0.0,0,0
3,1,NaN,4,5
4,1,2.0,2,5
5,1,2.0,2,5


In [ ]:
df['skuska'] = df.groupby(["a"])['c'].rolling(5, min_periods=1).mean().values
df

,a,b,c,d,skuska
0,1,2.0,3,4,3.00
1,1,NaN,4,5,3.50
2,2,1.0,3,5,3.00
3,1,2.0,2,5,2.75
4,1,2.0,2,5,2.60
5,1,2.0,2,5,3.00


In [ ]:
df['skuska'] = df.groupby(["a"])['skuska'].shift(fill_value=0)
df

,a,b,c,d,skuska
0,1,2.0,3,4,0.00
1,1,NaN,4,5,3.00
2,2,1.0,3,5,0.00
3,1,2.0,2,5,3.50
4,1,2.0,2,5,2.75
5,1,2.0,2,5,2.60


In [ ]:
df['skuska'] = df.groupby(["a"])['skuska'].shift(fill_value=0)
df

KeyError: ignored